# Inspect lenses and generate GLASS configs

## Imports

In [ ]:
import sys
import os
import pprint
import json
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets

from astropy.io import fits
from astropy.wcs import WCS
from astropy.nddata import Cutout2D

root = "/Users/phdenzel/gleam"
sys.path.append(root)
import gleam
from gleam.lensobject import LensObject
from gleam.multilens import MultiLens
from gleam.utils.plotting import IPColorbar, IPPointCache
import gleam.utils.colors as gcl
gcl.GLEAMcmaps.register_all()

## Main

### Search for a fits file

In [ ]:
fidx = 0
files = !ls data/*[0-9].fits
pprint.pprint(files)
filename = files[fidx]
lensname = os.path.basename(filename).split('.')[0]
print("Selected file: {}".format(lensname))

### Read fits file

In [ ]:
hdu = fits.open(filename)
hdu[0].header

In [ ]:
lo = LensObject(filename, auto=False, glscfactory_options={})
print(lo.__v__)

In [ ]:
if 'ORIENTAT' in lo.hdr:
    orientat = lo.hdr['ORIENTAT']
else:
    orientat = 0
orientat

### Define lensed images

In [ ]:
%matplotlib notebook

fig, ax = plt.subplots()
fig, ax, plt_out = lo.plot_f(fig, ax=ax, cmap=gcl.GLEAMcmaps.vilux, colorbar=True,
                             vmin=0, vmax=0.5*np.max(lo.data), source_images=False,
                             deconv=False, psf=np.ones((3, 3))/25.)
clrbar = IPColorbar(plt_out[1], plt_out[0])
clrbar.connect()
cache = IPPointCache(plt_out[0])
cid = cache.connect()

In [ ]:
if cache.xy:
    subj = cache.modes.index('L')
    lens_pos = cache.xy[subj]
    limg_pos = cache.xy[:subj] + cache.xy[subj+1:]
    
    print(lens_pos)
    print(limg_pos)

In [ ]:
# add lens and lensed image positions to lens object
print(lens_pos)
print(limg_pos)
# lo.lens = lo.center
# for img in info['srcimgs']:
#     lo.add_srcimg(lo.lens.xy+img/lo.px2arcsec, unit='pixel', relative=True)

In [ ]:
%matplotlib notebook

fig, ax = plt.subplots()
fig, ax, plt_out = lo.plot_f(fig, ax=ax, cmap=gcl.GLEAMcmaps.vilux, colorbar=True,
                             vmin=0, vmax=0.5*np.max(lo.data), source_images=True,
                             deconv=False, psf=np.ones((3, 3))/25.)
clrbar = IPColorbar(plt_out[1], plt_out[0])
clrbar.connect()

In [ ]:
print(lo.__v__)

### Save GLASS configs

In [ ]:
%%script false
lo.glscfactory.sync_lens_params()
configs = lo.glscfactory.config['single']
print("".join(configs))

In [ ]:
%%script false
savename = "{}.gls".format(lensname)
lo.glscfactory.write(filename=savename)